In [1]:
from numpy import asarray
from numpy.random import randn
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt

def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    z_input = x_input.reshape(n_samples, latent_dim)
    print(z_input.shape)
    return z_input

def plot_generated(examples, n):
    plt.figure(figsize=(16, 16))
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(examples[i, :, :])
    plt.show()

#### Loading the generator model and generate 25 new images

In [3]:
model = load_model('../input/hiwitask/generator_model_128x128_100V2.h5')
latent_points = generate_latent_points(100, 9)
X = model.predict(latent_points)
X = (X + 1) / 2.0 
plot_generated(X, 3)


#### Interpolate in latent space

In [4]:
from numpy import linspace

def interpolate_points(p1, p2, n_steps=10):
    #interpolate between points (e.g., between 0 and 1 if you divde to 10 then you have 0.111, 0.222, etc.)
    ratios = linspace(0, 1, num=n_steps)
    
    # linear interpolation of vectors based on the above interpolation ratios
    vectors = list()
    for ratio in ratios:  
        v = (1.0 - ratio) * p1 + ratio * p2
        vectors.append(v)
    return asarray(vectors)

def plot_generated_interpolate(examples, n):
    # plot images
    plt.figure(figsize=(30, 30))
    for i in range(n):   
        plt.subplot(1, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :])
    plt.show()

In [5]:
pts = generate_latent_points(100, 2)

In [7]:

interpolated = interpolate_points(pts[0], pts[1], 6)
X = model.predict(interpolated)
X = (X + 1) / 2.0
plot_generated_interpolate(X, len(interpolated))

In [ ]:
# #Test Block

# diff = interpolated[0] - interpolated[9]
# random_pt = generate_latent_points(100, 1)
# random_pt_img = model.predict(random_pt)
# random_pt_img = (random_pt_img + 1) / 2.0
# plt.imshow(random_pt_img[0])



In [ ]:
# #Test Block

# changed_img = model.predict(diff + random_pt)
# changed_img = (changed_img + 1.0) / 2.0
# plt.imshow(changed_img[0])

#### Generate new images with specific features - using vector arithmetic operation

In [8]:
from numpy import mean, expand_dims
latent_points = generate_latent_points(100, 100)
X  = model.predict(latent_points)
X = (X + 1) / 2.0
plot_generated(X, 10)

In [9]:
# index of images
glasses = [16, 37, 83]
no_glasses = [91, 92, 89]
smiling = [11, 21, 71]

In [10]:
def average_points(points, ix):
    zero_ix = [i-1 for i in ix]
    vectors = points[zero_ix]
    avg_vector = mean(vectors, axis=0)
    return avg_vector

feature1 = average_points(latent_points, glasses)
feature2 = average_points(latent_points, no_glasses)
feature3 = average_points(latent_points, smiling)

### Average image with Glasses 

In [15]:
feature1_vector = expand_dims(feature1, 0)
feature1_image = model.predict(feature1_vector)
feature1_image = (feature1_image + 1) / 2.0
plt.imshow(feature1_image[0])
plt.show()

#### Average image of Withour Glasses

In [16]:
feature2_vector = expand_dims(feature2, 0)
feature2_image = model.predict(feature2_vector)
feature2_image = (feature2_image + 1) / 2.0
plt.imshow(feature2_image[0])
plt.show()

#### Average image of smiling faces

In [17]:
feature3_vector = expand_dims(feature3, 0)
feature3_image = model.predict(feature3_vector)
feature3_image = (feature3_image + 1) / 2.0
plt.imshow(feature3_image[0])
plt.show()

#### Generating a face with smile and glasses

In [18]:
result_vector = feature1 - feature2 + feature3
result_vector = expand_dims(result_vector, 0)
result_image = model.predict(result_vector)
result_image = (result_image + 1) / 2.0
plt.imshow(result_image[0])
plt.show()